<a href="https://colab.research.google.com/github/mcui5/dl-final/blob/main/CNNLSTM_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf 
from tensorflow.keras import Model
import numpy as np
import pickle 
import os
import pandas as pd 


from google.colab import drive #Can ignore if done locally
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def preprocess(filepath): 
  """
    1. Unpickle file
    2. Separate 
    3. One-hot encode labels 

    :inputs: 
    filepath: filepath to the pickle file in Drive 

    :returns: 
    (inputs, labels, folders)
  """
  
  with open(filepath, 'rb') as fo:
    pickle_output = pickle.load(fo, encoding='bytes')
  
  inputs = [row[0] for row in pickle_output]
  inputs = [inputs[i][:173] for i in range(len(inputs))]
  inputs = np.array(inputs)
  labels = np.array(pickle_output)[:, 1]
  folders = np.array(pickle_output)[:, 2]

  return (inputs, labels, folders)

def split(inputs, labels, folders, test_folder_idx):
  """
    Split data into training and testing data 

    :inputs: 
    the outputs from preprocess 
    test_folder_idx: index of the folder that will be used for testing

    :return: 
    one quadruple, (train_inputs, train_labels, test_inputs, test_labels)
  """
  test_indices = np.nonzero(folders == test_folder_idx)
  train_indices = np.nonzero(folders != test_folder_idx)

  return (inputs[train_indices], labels[train_indices], inputs[test_indices], labels[test_indices])

In [ ]:
def shuffle(inputs, labels, test_fraction):
  '''
  shuffle collection of all data, and split into testing and training, 15%:85%

  :inputs: 
    the outputs from preprocess (inputs and labels)
    test_fraction: percentage of inputs that will be used for testing
  
  :return: 
    one quadruple, (train_inputs, train_labels, test_inputs, test_labels)
  '''
  indices = np.arange(labels.shape[0])
  np.random.shuffle(indices)
  inputs = np.take(inputs, indices, axis=0)
  labels = np.take(labels, indices, axis=0)

  num_test = int(test_fraction * labels.shape[0])
  test_inputs = inputs[:num_test]
  test_labels = labels[:num_test]
  train_inputs = inputs[num_test:]
  train_labels = labels[num_test:]

  return (train_inputs, train_labels, test_inputs, test_labels)

In [ ]:
class CNNLSTMModel(tf.keras.Model):

  def __init__(self, num_batches):
    super(CNNLSTMModel, self).__init__()

    self.lstm_dropout = 0.2
    self.dropout_rate = 0.8 #0.25
    self.lstm_size = 256
    self.batch_size = num_batches

    #adam optimizer
    self.optimizer = tf.keras.optimizers.Adam(lr = 1e-4)

    #initialize layers
    self.lstm1 = tf.keras.layers.LSTM(self.lstm_size, dropout=self.lstm_dropout)
    self.dense1 = tf.keras.layers.Dense(10, activation='softmax')

    self.conv1 = tf.keras.layers.Conv2D(filters=4, kernel_size=(5,5), strides=(4,1), activation='relu')
    self.conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), strides=(2,1), activation='relu')
    self.conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=(2,2), strides=(2,1), activation='relu')
   
    self.conv4 = tf.keras.layers.Conv2D(filters=300, kernel_size=(2,2), strides=(1,1), activation='relu')

    self.maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=(3, 1), strides=(2,1))
    self.maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=(3, 1), strides=(2,1))

    self.dropout1 = tf.keras.layers.Dropout(self.dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(self.dropout_rate)
    self.dropout3 = tf.keras.layers.Dropout(self.dropout_rate)
    
  def call(self, inputs):
    #padding='same' if this doesn't work


    inputs = tf.expand_dims(inputs, axis=3)


    convlayer1 = self.conv1(inputs)
    # convlayer2 = self.conv2(convlayer1)
    maxpool1 = self.maxpool1(convlayer1)

    drop1 = self.dropout1(maxpool1)
    convlayer3 = self.conv3(drop1)
    maxpool2 = self.maxpool2(convlayer3)

    drop2 = self.dropout2(maxpool2)
    convlayer4 = self.conv4(drop2)
    drop3 = self.dropout3(convlayer4)

    # reshape = tf.reshape(drop3, (100,300,32))
    reshape = tf.reshape(drop3, (100,300,102))

    lstm = self.lstm1(reshape)
    dense = self.dense1(lstm)

    return dense

  def loss(self, logits, labels):

    '''As cited in the paper, Table IV Experimental Results, authors used Categorical
      cross entropy loss to measure their CNN+LSTM models'''
    #return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, logits))
    losses = tf.keras.losses.categorical_crossentropy(labels, logits, from_logits=True)
    return tf.reduce_mean(losses)

  def accuracy(self, logits, labels):

    correct_predictions = tf.equal(tf.argmax(logits, 1), tf.argmax(labels,1))
    return tf.reduce_sum(tf.cast(correct_predictions, tf.float32))

In [ ]:
def train(model, train_inputs, train_labels):
  for i in range(len(train_inputs) // model.batch_size):
      # getting the proper batch 
      start = i * model.batch_size 
      inputs = train_inputs[start : start + model.batch_size]
      labels = train_labels[start : start + model.batch_size]


      with tf.GradientTape() as tape:
          # forward pass 
          logits = model.call(inputs)
          loss = model.loss(logits, labels)
                  
      # backprop 
      gradients = tape.gradient(loss, model.trainable_variables)
      model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

def test(model, test_inputs, test_labels): 
  """
    returns the TOTAL accuracy for a single FOLDER 
  """
  num_batches = len(test_inputs) // model.batch_size
  total_right = 0

  for i in range(num_batches):
      # getting the proper batch 
      start = i * model.batch_size 
      inputs = test_inputs[start : start + model.batch_size]
      labels = test_labels[start : start + model.batch_size]

      # calling the model to get our probabilities 
      logits = model.call(inputs)
      total_right += model.accuracy(logits, labels)
  
  return total_right

In [ ]:
# pickled_path = '/content/gdrive/Shared drives/CS1470-Final/mfccs.pkl'
# inputs, labels, folders = preprocess(pickled_path)

# batch_size, num_epochs = 100, 250
  
# accuracy = 0 
# total_tested = 0 
# for i in range(10):
#   print("Split/test folder: ", i + 1) 
#   model = CNNLSTMModel(batch_size)
#   tr_in, tr_lb, te_in, te_lb = split(inputs, labels, folders, i + 1)
#   tr_in = tf.convert_to_tensor(tr_in, dtype=tf.float32)
#   te_in = tf.convert_to_tensor(te_in, dtype=tf.float32)
#   tr_lb = tf.one_hot(tr_lb, 10, dtype=tf.int64)
#   te_lb = tf.one_hot(te_lb, 10, dtype=tf.int64)

#   for i in range(num_epochs): 
#     train(model, tr_in, tr_lb)
#     print(i)

#   per_fold_acc = test(model, te_in, te_lb)
#   accuracy += per_fold_acc
#   print('Cumulative right: ' + str(accuracy))
     
#   per_fold_tested = (len(te_lb) - (len(te_lb) % batch_size))
#   total_tested += per_fold_tested
#   print('per-fold acc: ' + str(per_fold_acc / per_fold_tested))
  
# print("Total Average Accuracy: ", accuracy / total_tested)

In [ ]:
'''
Shuffled version: extracted 15% for testing (follows paper)
'''
pickled_path = '/content/gdrive/Shared drives/CS1470-Final/mfccs.pkl'
inputs, labels, folders = preprocess(pickled_path)

batch_size, num_epochs = 100, 500
  
model = CNNLSTMModel(batch_size)
print(np.shape(inputs))
tr_in, tr_lb, te_in, te_lb = shuffle(inputs, labels, 0.15)

tr_in = tf.convert_to_tensor(tr_in, dtype=tf.float32)
te_in = tf.convert_to_tensor(te_in, dtype=tf.float32)
tr_lb = tf.one_hot(tr_lb, 10, dtype=tf.int64)
te_lb = tf.one_hot(te_lb, 10, dtype=tf.int64)

for i in range(num_epochs): 
  train(model, tr_in, tr_lb)
  print(i)
acc = test(model, te_in, te_lb)
print(np.shape(te_in))
print(np.shape(te_lb))
print("Average Accuracy: ", acc / te_lb.shape[0])

(6573, 173, 40)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
27